In [119]:
#import dependencies
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization, Input, Concatenate, TimeDistributed, Flatten, Lambda, Reshape
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.regularizers import l2

In [120]:
df = pd.read_csv('nbaPreProcessed.csv')
df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_points_1OT,homeTeam_points_2OT,homeTeam_points_3OT,homeTeam_points_4OT,homeTeam_FG,homeTeam_FG_made,homeTeam_3P,homeTeam_3P_made,homeTeam_O_Reb,homeTeam_D_Reb,homeTeam_Total_Reb,homeTeam_Ast,homeTeam_Stl,homeTeam_Blk,homeTeam_TO,homeTeam_PF,awayTeam,awayTeam_id,awayTeamSubject_id,awayTeam_win%,awayTeam_wins,awayTeam_losses,awayTeam_points_total,awayTeam_points_q1,awayTeam_points_q2,awayTeam_points_q3,awayTeam_points_q4,awayTeam_points_1OT,awayTeam_points_2OT,awayTeam_points_3OT,awayTeam_points_4OT,awayTeam_FG,awayTeam_FG_made,awayTeam_3P,awayTeam_3P_made,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame,winner_binary
0,2024-06-17,Boston Celtics,BOS,2,0.792079,80,21,106,28,39,19,20,0,0,0,0,0.426966,38,0.333333,13,15,36,51,25,9,2,7,15,Dallas Mavericks,DAL,7,0.605769,63,41,88,18,28,21,21,0,0,0,0,0.448718,35,0.297297,11,7,28,35,18,4,4,13,20,Boston Celtics,2023-2024,True,0
1,2024-06-14,Dallas Mavericks,DAL,7,0.611650,63,40,122,34,27,31,30,0,0,0,0,0.505495,46,0.405405,15,13,39,52,21,7,2,8,17,Boston Celtics,BOS,2,0.790000,79,21,84,21,14,25,24,0,0,0,0,0.362500,29,0.341463,14,4,27,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True,0
2,2024-06-12,Dallas Mavericks,DAL,7,0.607843,62,40,99,31,20,19,29,0,0,0,0,0.441860,38,0.360000,9,7,36,43,15,5,1,8,17,Boston Celtics,BOS,2,0.797980,79,20,106,30,20,35,21,0,0,0,0,0.463415,38,0.369565,17,6,30,36,26,4,6,9,19,Boston Celtics,2023-2024,True,1
3,2024-06-09,Boston Celtics,BOS,2,0.795918,78,20,105,25,29,29,22,0,0,0,0,0.452381,38,0.256410,10,10,31,41,29,10,5,10,15,Dallas Mavericks,DAL,7,0.613861,62,39,98,28,23,23,24,0,0,0,0,0.475000,38,0.230769,6,9,34,43,21,5,3,15,17,Boston Celtics,2023-2024,True,0
4,2024-06-06,Boston Celtics,BOS,2,0.793814,77,20,107,37,26,23,21,0,0,0,0,0.475610,39,0.380952,16,10,37,47,23,6,9,12,16,Dallas Mavericks,DAL,7,0.620000,62,38,89,20,22,24,23,0,0,0,0,0.416667,35,0.259259,7,10,33,43,9,8,1,11,16,Boston Celtics,2023-2024,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,1,104,21,19,27,18,9,10,0,0,0.425287,37,0.375000,3,11,34,45,23,3,8,14,24,Indiana Pacers,IND,12,1.000000,1,0,109,26,20,17,22,9,15,0,0,0.462366,43,0.235294,4,11,36,47,24,9,5,13,28,Indiana Pacers,2004-2005,False,1
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,1,95,22,31,30,12,0,0,0,0,0.478261,33,0.294118,5,10,25,35,23,9,3,19,27,Philadelphia 76ers,PHI,23,1.000000,1,0,98,27,24,20,27,0,0,0,0,0.444444,36,0.375000,9,14,27,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False,1
25728,2004-11-02,Los Angeles Lakers,LAL,14,1.000000,1,0,89,24,26,22,17,0,0,0,0,0.428571,30,0.294118,5,10,37,47,23,8,9,19,22,Denver Nuggets,DEN,8,0.000000,0,1,78,14,17,26,21,0,0,0,0,0.340909,30,0.250000,3,16,32,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False,0
25729,2004-11-02,Detroit Pistons,DET,9,1.000000,1,0,87,19,18,22,28,0,0,0,0,0.444444,32,0.333333,6,13,29,42,20,9,5,12,22,Houston Rockets,HOU,11,0.000000,0,1,79,18,20,21,20,0,0,0,0,0.394366,28,0.461538,6,9,27,36,8,4,5,16,24,Detroit Pistons,2004-2005,False,0


In [121]:
df.columns

Index(['date', 'homeTeam', 'homeTeam_id', 'homeTeamSubject_id',
       'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses',
       'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2',
       'homeTeam_points_q3', 'homeTeam_points_q4', 'homeTeam_points_1OT',
       'homeTeam_points_2OT', 'homeTeam_points_3OT', 'homeTeam_points_4OT',
       'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made',
       'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb',
       'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO',
       'homeTeam_PF', 'awayTeam', 'awayTeam_id', 'awayTeamSubject_id',
       'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses',
       'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2',
       'awayTeam_points_q3', 'awayTeam_points_q4', 'awayTeam_points_1OT',
       'awayTeam_points_2OT', 'awayTeam_points_3OT', 'awayTeam_points_4OT',
       'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made',
    

In [122]:
#populate dfIDtoStat - should be double the size of df
homedf = df[['date', 'homeTeam', 'homeTeamSubject_id', 'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses', 'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4', 'homeTeam_points_1OT', 'homeTeam_points_2OT', 'homeTeam_points_3OT', 'homeTeam_points_4OT', 'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO', 'homeTeam_PF', 'awayTeamSubject_id', 'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 'awayTeam_points_q4', 'awayTeam_points_1OT', 'awayTeam_points_2OT', 'awayTeam_points_3OT', 'awayTeam_points_4OT', 'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF', 'season', 'isPlayoffGame', 'winner']]
homedf.columns = ['date', 'team', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_total', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_1OT', 'team_points_2OT', 'team_points_3OT', 'team_points_4OT', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF', 'opponent_id', 'opponent_win%', 'opponent_wins', 'opponent_losses', 'opponent_points_total', 'opponent_points_q1', 'opponent_points_q2', 'opponent_points_q3', 'opponent_points_q4', 'opponent_points_1OT', 'opponent_points_2OT', 'opponent_points_3OT', 'opponent_points_4OT', 'opponent_FG', 'opponent_FG_made', 'opponent_3P', 'opponent_3P_made', 'opponent_O_Reb', 'opponent_D_Reb', 'opponent_Total_Reb', 'opponent_Ast', 'opponent_Stl', 'opponent_Blk', 'opponent_TO', 'opponent_PF', 'season', 'isPlayoffGame', 'winner']
awaydf = df[['date', 'awayTeam', 'awayTeamSubject_id', 'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 'awayTeam_points_q4', 'awayTeam_points_1OT', 'awayTeam_points_2OT', 'awayTeam_points_3OT', 'awayTeam_points_4OT', 'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF', 'homeTeamSubject_id', 'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses', 'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4', 'homeTeam_points_1OT', 'homeTeam_points_2OT', 'homeTeam_points_3OT', 'homeTeam_points_4OT', 'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO', 'homeTeam_PF', 'season', 'isPlayoffGame', 'winner']]
awaydf.columns = ['date', 'team', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_total', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_1OT', 'team_points_2OT', 'team_points_3OT', 'team_points_4OT', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF', 'opponent_id', 'opponent_win%', 'opponent_wins', 'opponent_losses', 'opponent_points_total', 'opponent_points_q1', 'opponent_points_q2', 'opponent_points_q3', 'opponent_points_q4', 'opponent_points_1OT', 'opponent_points_2OT', 'opponent_points_3OT', 'opponent_points_4OT', 'opponent_FG', 'opponent_FG_made', 'opponent_3P', 'opponent_3P_made', 'opponent_O_Reb', 'opponent_D_Reb', 'opponent_Total_Reb', 'opponent_Ast', 'opponent_Stl', 'opponent_Blk', 'opponent_TO', 'opponent_PF', 'season', 'isPlayoffGame', 'winner']
dfIDtoStat = pd.concat([homedf, awaydf], ignore_index=True)
dfIDtoStat

,date,team,team_id,team_win%,team_wins,team_losses,team_points_total,team_points_q1,team_points_q2,team_points_q3,team_points_q4,team_points_1OT,team_points_2OT,team_points_3OT,team_points_4OT,team_FG,team_FG_made,team_3P,team_3P_made,team_O_Reb,team_D_Reb,team_Total_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PF,opponent_id,opponent_win%,opponent_wins,opponent_losses,opponent_points_total,opponent_points_q1,opponent_points_q2,opponent_points_q3,opponent_points_q4,opponent_points_1OT,opponent_points_2OT,opponent_points_3OT,opponent_points_4OT,opponent_FG,opponent_FG_made,opponent_3P,opponent_3P_made,opponent_O_Reb,opponent_D_Reb,opponent_Total_Reb,opponent_Ast,opponent_Stl,opponent_Blk,opponent_TO,opponent_PF,season,isPlayoffGame,winner
0,2024-06-17,Boston Celtics,2,0.792079,80,21,106,28,39,19,20,0,0,0,0,0.426966,38,0.333333,13,15,36,51,25,9,2,7,15,7,0.605769,63,41,88,18,28,21,21,0,0,0,0,0.448718,35,0.297297,11,7,28,35,18,4,4,13,20,2023-2024,True,Boston Celtics
1,2024-06-14,Dallas Mavericks,7,0.611650,63,40,122,34,27,31,30,0,0,0,0,0.505495,46,0.405405,15,13,39,52,21,7,2,8,17,2,0.790000,79,21,84,21,14,25,24,0,0,0,0,0.362500,29,0.341463,14,4,27,31,18,2,5,13,19,2023-2024,True,Dallas Mavericks
2,2024-06-12,Dallas Mavericks,7,0.607843,62,40,99,31,20,19,29,0,0,0,0,0.441860,38,0.360000,9,7,36,43,15,5,1,8,17,2,0.797980,79,20,106,30,20,35,21,0,0,0,0,0.463415,38,0.369565,17,6,30,36,26,4,6,9,19,2023-2024,True,Boston Celtics
3,2024-06-09,Boston Celtics,2,0.795918,78,20,105,25,29,29,22,0,0,0,0,0.452381,38,0.256410,10,10,31,41,29,10,5,10,15,7,0.613861,62,39,98,28,23,23,24,0,0,0,0,0.475000,38,0.230769,6,9,34,43,21,5,3,15,17,2023-2024,True,Boston Celtics
4,2024-06-06,Boston Celtics,2,0.793814,77,20,107,37,26,23,21,0,0,0,0,0.475610,39,0.380952,16,10,37,47,23,6,9,12,16,7,0.620000,62,38,89,20,22,24,23,0,0,0,0,0.416667,35,0.259259,7,10,33,43,9,8,1,11,16,2023-2024,True,Boston Celtics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,Indiana Pacers,12,1.000000,1,0,109,26,20,17,22,9,15,0,0,0.462366,43,0.235294,4,11,36,47,24,9,5,13,28,6,0.000000,0,1,104,21,19,27,18,9,10,0,0,0.425287,37,0.375000,3,11,34,45,23,3,8,14,24,2004-2005,False,Indiana Pacers
51458,2004-11-03,Philadelphia 76ers,23,1.000000,1,0,98,27,24,20,27,0,0,0,0,0.444444,36,0.375000,9,14,27,41,20,9,2,16,26,2,0.000000,0,1,95,22,31,30,12,0,0,0,0,0.478261,33,0.294118,5,10,25,35,23,9,3,19,27,2004-2005,False,Philadelphia 76ers
51459,2004-11-02,Denver Nuggets,8,0.000000,0,1,78,14,17,26,21,0,0,0,0,0.340909,30,0.250000,3,16,32,48,17,10,8,16,21,14,1.000000,1,0,89,24,26,22,17,0,0,0,0,0.428571,30,0.294118,5,10,37,47,23,8,9,19,22,2004-2005,False,Los Angeles Lakers
51460,2004-11-02,Houston Rockets,11,0.000000,0,1,79,18,20,21,20,0,0,0,0,0.394366,28,0.461538,6,9,27,36,8,4,5,16,24,9,1.000000,1,0,87,19,18,22,28,0,0,0,0,0.444444,32,0.333333,6,13,29,42,20,9,5,12,22,2004-2005,False,Detroit Pistons


In [123]:
for idx, col in enumerate(dfIDtoStat.columns):
    print(f"Index {idx}: Column '{col}'")

Index 0: Column 'date'
Index 1: Column 'team'
Index 2: Column 'team_id'
Index 3: Column 'team_win%'
Index 4: Column 'team_wins'
Index 5: Column 'team_losses'
Index 6: Column 'team_points_total'
Index 7: Column 'team_points_q1'
Index 8: Column 'team_points_q2'
Index 9: Column 'team_points_q3'
Index 10: Column 'team_points_q4'
Index 11: Column 'team_points_1OT'
Index 12: Column 'team_points_2OT'
Index 13: Column 'team_points_3OT'
Index 14: Column 'team_points_4OT'
Index 15: Column 'team_FG'
Index 16: Column 'team_FG_made'
Index 17: Column 'team_3P'
Index 18: Column 'team_3P_made'
Index 19: Column 'team_O_Reb'
Index 20: Column 'team_D_Reb'
Index 21: Column 'team_Total_Reb'
Index 22: Column 'team_Ast'
Index 23: Column 'team_Stl'
Index 24: Column 'team_Blk'
Index 25: Column 'team_TO'
Index 26: Column 'team_PF'
Index 27: Column 'opponent_id'
Index 28: Column 'opponent_win%'
Index 29: Column 'opponent_wins'
Index 30: Column 'opponent_losses'
Index 31: Column 'opponent_points_total'
Index 32: 

In [124]:
one_hot = pd.get_dummies(dfIDtoStat['team_id'], prefix='team',  dtype=int)
dfIDtoStat = pd.concat([dfIDtoStat.iloc[:, :27], one_hot, dfIDtoStat.iloc[:, 27:]], axis=1)


dfIDtoStat

,date,team,team_id,team_win%,team_wins,team_losses,team_points_total,team_points_q1,team_points_q2,team_points_q3,team_points_q4,team_points_1OT,team_points_2OT,team_points_3OT,team_points_4OT,team_FG,team_FG_made,team_3P,team_3P_made,team_O_Reb,team_D_Reb,team_Total_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PF,team_1,team_2,team_3,team_4,team_5,team_6,team_7,team_8,team_9,team_10,team_11,team_12,team_13,team_14,team_15,team_16,team_17,team_18,team_19,team_20,team_21,team_22,team_23,team_24,team_25,team_26,team_27,team_28,team_29,team_30,opponent_id,opponent_win%,opponent_wins,opponent_losses,opponent_points_total,opponent_points_q1,opponent_points_q2,opponent_points_q3,opponent_points_q4,opponent_points_1OT,opponent_points_2OT,opponent_points_3OT,opponent_points_4OT,opponent_FG,opponent_FG_made,opponent_3P,opponent_3P_made,opponent_O_Reb,opponent_D_Reb,opponent_Total_Reb,opponent_Ast,opponent_Stl,opponent_Blk,opponent_TO,opponent_PF,season,isPlayoffGame,winner
0,2024-06-17,Boston Celtics,2,0.792079,80,21,106,28,39,19,20,0,0,0,0,0.426966,38,0.333333,13,15,36,51,25,9,2,7,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0.605769,63,41,88,18,28,21,21,0,0,0,0,0.448718,35,0.297297,11,7,28,35,18,4,4,13,20,2023-2024,True,Boston Celtics
1,2024-06-14,Dallas Mavericks,7,0.611650,63,40,122,34,27,31,30,0,0,0,0,0.505495,46,0.405405,15,13,39,52,21,7,2,8,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.790000,79,21,84,21,14,25,24,0,0,0,0,0.362500,29,0.341463,14,4,27,31,18,2,5,13,19,2023-2024,True,Dallas Mavericks
2,2024-06-12,Dallas Mavericks,7,0.607843,62,40,99,31,20,19,29,0,0,0,0,0.441860,38,0.360000,9,7,36,43,15,5,1,8,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.797980,79,20,106,30,20,35,21,0,0,0,0,0.463415,38,0.369565,17,6,30,36,26,4,6,9,19,2023-2024,True,Boston Celtics
3,2024-06-09,Boston Celtics,2,0.795918,78,20,105,25,29,29,22,0,0,0,0,0.452381,38,0.256410,10,10,31,41,29,10,5,10,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0.613861,62,39,98,28,23,23,24,0,0,0,0,0.475000,38,0.230769,6,9,34,43,21,5,3,15,17,2023-2024,True,Boston Celtics
4,2024-06-06,Boston Celtics,2,0.793814,77,20,107,37,26,23,21,0,0,0,0,0.475610,39,0.380952,16,10,37,47,23,6,9,12,16,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0.620000,62,38,89,20,22,24,23,0,0,0,0,0.416667,35,0.259259,7,10,33,43,9,8,1,11,16,2023-2024,True,Boston Celtics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,Indiana Pacers,12,1.000000,1,0,109,26,20,17,22,9,15,0,0,0.462366,43,0.235294,4,11,36,47,24,9,5,13,28,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0.000000,0,1,104,21,19,27,18,9,10,0,0,0.425287,37,0.375000,3,11,34,45,23,3,8,14,24,2004-2005,False,Indiana Pacers
51458,2004-11-03,Philadelphia 76ers,23,1.000000,1,0,98,27,24,20,27,0,0,0,0,0.444444,36,0.375000,9,14,27,41,20,9,2,16,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0.000000,0,1,95,22,31,30,12,0,0,0,0,0.478261,33,0.294118,5,10,25,35,23,9,3,19,27,2004-2005,False,Philadelphia 76ers
51459,2004-11-02,Denver Nuggets,8,0.000000,0,1,78,14,17,26,21,0,0,0,0,0.340909,30,0.250000,3,16,32,48,17,10,8,16,21,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,1.000000,1,0,89,24,26,22,17,0,0,0,0,0.428571,30,0.294118,5,10,37,47,23,8,9,19,22,2004-2005,False,Los Angeles Lakers
51460,2004-11-02,Houston Rockets,11,0.000000,0,1,79,18,20,21,20,0,0,0,0,0.394366,28,0.461538,6,9,27,36,8,4,5,16,24,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,1.000000,1,0,87,19,18,22,28,0,0,0,0,0.444444,32,0.333333,6,13,29,42,20,9,5,12,22,2004-2005,False,Detroit Pistons


In [125]:
for idx, col in enumerate(dfIDtoStat.columns):
    print(f"Index {idx}: Column '{col}'")

Index 0: Column 'date'
Index 1: Column 'team'
Index 2: Column 'team_id'
Index 3: Column 'team_win%'
Index 4: Column 'team_wins'
Index 5: Column 'team_losses'
Index 6: Column 'team_points_total'
Index 7: Column 'team_points_q1'
Index 8: Column 'team_points_q2'
Index 9: Column 'team_points_q3'
Index 10: Column 'team_points_q4'
Index 11: Column 'team_points_1OT'
Index 12: Column 'team_points_2OT'
Index 13: Column 'team_points_3OT'
Index 14: Column 'team_points_4OT'
Index 15: Column 'team_FG'
Index 16: Column 'team_FG_made'
Index 17: Column 'team_3P'
Index 18: Column 'team_3P_made'
Index 19: Column 'team_O_Reb'
Index 20: Column 'team_D_Reb'
Index 21: Column 'team_Total_Reb'
Index 22: Column 'team_Ast'
Index 23: Column 'team_Stl'
Index 24: Column 'team_Blk'
Index 25: Column 'team_TO'
Index 26: Column 'team_PF'
Index 27: Column 'team_1'
Index 28: Column 'team_2'
Index 29: Column 'team_3'
Index 30: Column 'team_4'
Index 31: Column 'team_5'
Index 32: Column 'team_6'
Index 33: Column 'team_7'
I

In [126]:
one_hot = pd.get_dummies(dfIDtoStat['opponent_id'], prefix='opponent',  dtype=int)
dfIDtoStat = pd.concat([dfIDtoStat.iloc[:, :82], one_hot, dfIDtoStat.iloc[:, 82:]], axis=1)
dfIDtoStat= dfIDtoStat.drop('opponent_id', axis=1)

In [127]:
for idx, col in enumerate(dfIDtoStat.columns):
    print(f"Index {idx}: Column '{col}'")

Index 0: Column 'date'
Index 1: Column 'team'
Index 2: Column 'team_id'
Index 3: Column 'team_win%'
Index 4: Column 'team_wins'
Index 5: Column 'team_losses'
Index 6: Column 'team_points_total'
Index 7: Column 'team_points_q1'
Index 8: Column 'team_points_q2'
Index 9: Column 'team_points_q3'
Index 10: Column 'team_points_q4'
Index 11: Column 'team_points_1OT'
Index 12: Column 'team_points_2OT'
Index 13: Column 'team_points_3OT'
Index 14: Column 'team_points_4OT'
Index 15: Column 'team_FG'
Index 16: Column 'team_FG_made'
Index 17: Column 'team_3P'
Index 18: Column 'team_3P_made'
Index 19: Column 'team_O_Reb'
Index 20: Column 'team_D_Reb'
Index 21: Column 'team_Total_Reb'
Index 22: Column 'team_Ast'
Index 23: Column 'team_Stl'
Index 24: Column 'team_Blk'
Index 25: Column 'team_TO'
Index 26: Column 'team_PF'
Index 27: Column 'team_1'
Index 28: Column 'team_2'
Index 29: Column 'team_3'
Index 30: Column 'team_4'
Index 31: Column 'team_5'
Index 32: Column 'team_6'
Index 33: Column 'team_7'
I

In [128]:
one_hot = pd.get_dummies(dfIDtoStat['season'], prefix='season',  dtype=int)
dfIDtoStat = pd.concat([dfIDtoStat.iloc[:, :111], one_hot, dfIDtoStat.iloc[:, 111:]], axis=1)
dfIDtoStat = dfIDtoStat.drop('season', axis=1)


In [129]:
for idx, col in enumerate(dfIDtoStat.columns):
    print(f"Index {idx}: Column '{col}'")

Index 0: Column 'date'
Index 1: Column 'team'
Index 2: Column 'team_id'
Index 3: Column 'team_win%'
Index 4: Column 'team_wins'
Index 5: Column 'team_losses'
Index 6: Column 'team_points_total'
Index 7: Column 'team_points_q1'
Index 8: Column 'team_points_q2'
Index 9: Column 'team_points_q3'
Index 10: Column 'team_points_q4'
Index 11: Column 'team_points_1OT'
Index 12: Column 'team_points_2OT'
Index 13: Column 'team_points_3OT'
Index 14: Column 'team_points_4OT'
Index 15: Column 'team_FG'
Index 16: Column 'team_FG_made'
Index 17: Column 'team_3P'
Index 18: Column 'team_3P_made'
Index 19: Column 'team_O_Reb'
Index 20: Column 'team_D_Reb'
Index 21: Column 'team_Total_Reb'
Index 22: Column 'team_Ast'
Index 23: Column 'team_Stl'
Index 24: Column 'team_Blk'
Index 25: Column 'team_TO'
Index 26: Column 'team_PF'
Index 27: Column 'team_1'
Index 28: Column 'team_2'
Index 29: Column 'team_3'
Index 30: Column 'team_4'
Index 31: Column 'team_5'
Index 32: Column 'team_6'
Index 33: Column 'team_7'
I

In [130]:
dfIDtoStat['isPlayoffGame'] = dfIDtoStat['isPlayoffGame'].astype(int)
print(dfIDtoStat['isPlayoffGame'].unique())

[1 0]


In [131]:
dfIDtoStat['winner_boolean'] = (dfIDtoStat['winner'] == dfIDtoStat['team']).astype(int)
dfIDtoStat

,date,team,team_id,team_win%,team_wins,team_losses,team_points_total,team_points_q1,team_points_q2,team_points_q3,team_points_q4,team_points_1OT,team_points_2OT,team_points_3OT,team_points_4OT,team_FG,team_FG_made,team_3P,team_3P_made,team_O_Reb,team_D_Reb,team_Total_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PF,team_1,team_2,team_3,team_4,team_5,team_6,team_7,team_8,team_9,team_10,team_11,team_12,team_13,team_14,team_15,team_16,team_17,team_18,team_19,team_20,team_21,team_22,team_23,team_24,team_25,team_26,team_27,team_28,team_29,team_30,opponent_win%,opponent_wins,opponent_losses,opponent_points_total,opponent_points_q1,opponent_points_q2,opponent_points_q3,opponent_points_q4,opponent_points_1OT,opponent_points_2OT,opponent_points_3OT,opponent_points_4OT,opponent_FG,opponent_FG_made,opponent_3P,opponent_3P_made,opponent_O_Reb,opponent_D_Reb,opponent_Total_Reb,opponent_Ast,opponent_Stl,opponent_Blk,opponent_TO,opponent_PF,opponent_1,opponent_2,opponent_3,opponent_4,opponent_5,opponent_6,opponent_7,opponent_8,opponent_9,opponent_10,opponent_11,opponent_12,opponent_13,opponent_14,opponent_15,opponent_16,opponent_17,opponent_18,opponent_19,opponent_20,opponent_21,opponent_22,opponent_23,opponent_24,opponent_25,opponent_26,opponent_27,opponent_28,opponent_29,opponent_30,season_2004-2005,season_2005-2006,season_2006-2007,season_2007-2008,season_2008-2009,season_2009-2010,season_2010-2011,season_2011-2012,season_2012-2013,season_2013-2014,season_2014-2015,season_2015-2016,season_2016-2017,season_2017-2018,season_2018-2019,season_2019-2020,season_2020-2021,season_2021-2022,season_2022-2023,season_2023-2024,isPlayoffGame,winner,winner_boolean
0,2024-06-17,Boston Celtics,2,0.792079,80,21,106,28,39,19,20,0,0,0,0,0.426966,38,0.333333,13,15,36,51,25,9,2,7,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.605769,63,41,88,18,28,21,21,0,0,0,0,0.448718,35,0.297297,11,7,28,35,18,4,4,13,20,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,Boston Celtics,1
1,2024-06-14,Dallas Mavericks,7,0.611650,63,40,122,34,27,31,30,0,0,0,0,0.505495,46,0.405405,15,13,39,52,21,7,2,8,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.790000,79,21,84,21,14,25,24,0,0,0,0,0.362500,29,0.341463,14,4,27,31,18,2,5,13,19,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,Dallas Mavericks,1
2,2024-06-12,Dallas Mavericks,7,0.607843,62,40,99,31,20,19,29,0,0,0,0,0.441860,38,0.360000,9,7,36,43,15,5,1,8,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.797980,79,20,106,30,20,35,21,0,0,0,0,0.463415,38,0.369565,17,6,30,36,26,4,6,9,19,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,Boston Celtics,0
3,2024-06-09,Boston Celtics,2,0.795918,78,20,105,25,29,29,22,0,0,0,0,0.452381,38,0.256410,10,10,31,41,29,10,5,10,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.613861,62,39,98,28,23,23,24,0,0,0,0,0.475000,38,0.230769,6,9,34,43,21,5,3,15,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,Boston Celtics,1
4,2024-06-06,Boston Celtics,2,0.793814,77,20,107,37,26,23,21,0,0,0,0,0.475610,39,0.380952,16,10,37,47,23,6,9,12,16,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.620000,62,38,89,20,22,24,23,0,0,0,0,0.416667,35,0.259259,7,10,33,43,9,8,1,11,16,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,Boston Celtics,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [133]:
dfIDtoStat = dfIDtoStat.drop(columns=['team', 'winner'])
dfIDtoStat

,date,team_id,team_win%,team_wins,team_losses,team_points_total,team_points_q1,team_points_q2,team_points_q3,team_points_q4,team_points_1OT,team_points_2OT,team_points_3OT,team_points_4OT,team_FG,team_FG_made,team_3P,team_3P_made,team_O_Reb,team_D_Reb,team_Total_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PF,team_1,team_2,team_3,team_4,team_5,team_6,team_7,team_8,team_9,team_10,team_11,team_12,team_13,team_14,team_15,team_16,team_17,team_18,team_19,team_20,team_21,team_22,team_23,team_24,team_25,team_26,team_27,team_28,team_29,team_30,opponent_win%,opponent_wins,opponent_losses,opponent_points_total,opponent_points_q1,opponent_points_q2,opponent_points_q3,opponent_points_q4,opponent_points_1OT,opponent_points_2OT,opponent_points_3OT,opponent_points_4OT,opponent_FG,opponent_FG_made,opponent_3P,opponent_3P_made,opponent_O_Reb,opponent_D_Reb,opponent_Total_Reb,opponent_Ast,opponent_Stl,opponent_Blk,opponent_TO,opponent_PF,opponent_1,opponent_2,opponent_3,opponent_4,opponent_5,opponent_6,opponent_7,opponent_8,opponent_9,opponent_10,opponent_11,opponent_12,opponent_13,opponent_14,opponent_15,opponent_16,opponent_17,opponent_18,opponent_19,opponent_20,opponent_21,opponent_22,opponent_23,opponent_24,opponent_25,opponent_26,opponent_27,opponent_28,opponent_29,opponent_30,season_2004-2005,season_2005-2006,season_2006-2007,season_2007-2008,season_2008-2009,season_2009-2010,season_2010-2011,season_2011-2012,season_2012-2013,season_2013-2014,season_2014-2015,season_2015-2016,season_2016-2017,season_2017-2018,season_2018-2019,season_2019-2020,season_2020-2021,season_2021-2022,season_2022-2023,season_2023-2024,isPlayoffGame,winner_boolean
0,2024-06-17,2,0.792079,80,21,106,28,39,19,20,0,0,0,0,0.426966,38,0.333333,13,15,36,51,25,9,2,7,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.605769,63,41,88,18,28,21,21,0,0,0,0,0.448718,35,0.297297,11,7,28,35,18,4,4,13,20,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
1,2024-06-14,7,0.611650,63,40,122,34,27,31,30,0,0,0,0,0.505495,46,0.405405,15,13,39,52,21,7,2,8,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.790000,79,21,84,21,14,25,24,0,0,0,0,0.362500,29,0.341463,14,4,27,31,18,2,5,13,19,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
2,2024-06-12,7,0.607843,62,40,99,31,20,19,29,0,0,0,0,0.441860,38,0.360000,9,7,36,43,15,5,1,8,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.797980,79,20,106,30,20,35,21,0,0,0,0,0.463415,38,0.369565,17,6,30,36,26,4,6,9,19,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,2024-06-09,2,0.795918,78,20,105,25,29,29,22,0,0,0,0,0.452381,38,0.256410,10,10,31,41,29,10,5,10,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.613861,62,39,98,28,23,23,24,0,0,0,0,0.475000,38,0.230769,6,9,34,43,21,5,3,15,17,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
4,2024-06-06,2,0.793814,77,20,107,37,26,23,21,0,0,0,0,0.475610,39,0.380952,16,10,37,47,23,6,9,12,16,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.620000,62,38,89,20,22,24,23,0,0,0,0,0.416667,35,0.259259,7,10,33,43,9,8,1,11,16,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,12,1.000000,1,0,109,26,20,17,22,9,15,0,0,0.462366,43,0.235294,4,11,36,

In [134]:
#generate the X Y matrices
def df_to_X_y(df, dfWindow, window_size):
    X = []
    y = []
    #each team must have played window_size games before data can be extracted
    for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfWindow[dfWindow['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
            homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            homeTeam_window = homeTeam_window.to_numpy()
            awayTeam_window = awayTeam_window.to_numpy()
            combined_window = np.stack((homeTeam_window, awayTeam_window), axis=1)
            X.append(combined_window)
            y.append(row['winner_binary'])
    
    return np.array(X), np.array(y)

In [135]:
#X1.Shape is num data, window size, variables
X, y = df_to_X_y(df, dfIDtoStat, 20)
X.shape, y.shape

((25406, 20, 2, 130), (25406,))

In [136]:
#80-10-10 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((20324, 20, 2, 130),
 (20324,),
 (2541, 20, 2, 130),
 (2541,),
 (2541, 20, 2, 130),
 (2541,))

In [140]:
input_layer = Input(shape=(20,2,130))

home_team_input = input_layer[:, :, 0, :]
away_team_input = input_layer[:, :, 1, :]

home_team_lstm = LSTM(64, activation='tanh', return_sequences=True)(home_team_input)
home_team_lstm = LSTM(32, activation='tanh')(home_team_lstm)
home_team_lstm = BatchNormalization()(home_team_lstm)

away_team_lstm = LSTM(64, activation='tanh', return_sequences=True)(away_team_input)
away_team_lstm = LSTM(32, activation='tanh')(away_team_lstm)
away_team_lstm = BatchNormalization()(away_team_lstm)

combined = Concatenate()([home_team_lstm, away_team_lstm])

combined_flat = Flatten()(combined)

x = Dense(16, activation='relu')(combined_flat)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(8, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)

In [141]:
#early stopping if model does not improve performance for 50 epochs
earlystopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [142]:
#simple model fit
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, callbacks=[earlystopping])

Epoch 1/1000
636/636 [==============================] - 15s 15ms/step - loss: 0.7036 - accuracy: 0.5922 - val_loss: 2.1953 - val_accuracy: 0.4101
Epoch 2/1000
636/636 [==============================] - 8s 13ms/step - loss: 0.6523 - accuracy: 0.6280 - val_loss: 0.6445 - val_accuracy: 0.6411
Epoch 3/1000
636/636 [==============================] - 9s 13ms/step - loss: 0.6349 - accuracy: 0.6418 - val_loss: 0.6462 - val_accuracy: 0.6340
Epoch 4/1000
636/636 [==============================] - 8s 13ms/step - loss: 0.6525 - accuracy: 0.6227 - val_loss: 1.5736 - val_accuracy: 0.5903
Epoch 5/1000
636/636 [==============================] - 8s 13ms/step - loss: 0.6280 - accuracy: 0.6516 - val_loss: 0.6429 - val_accuracy: 0.6155
Epoch 6/1000
636/636 [==============================] - 8s 13ms/step - loss: 0.6235 - accuracy: 0.6544 - val_loss: 0.6270 - val_accuracy: 0.6505
Epoch 7/1000
636/636 [==============================] - 8s 13ms/step - loss: 0.6205 - accuracy: 0.6594 - val_loss: 1.2268 - val_a

In [143]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)
print(accuracy_score(y_test, y_pred_binary))

80/80 [==============================] - 1s 4ms/step
0.663124754033845
